## Job Web-Scraping Using Selenium
Note: This is intented for practice to extract real time job descriptions. Please adhere to the Robots.txt

In [28]:
#If you are using Selenium for the first time, please download the webdriver and note the filepath
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

#Used to allow the browser to load. 
import time
from tqdm import tqdm

#used as a import for dateing the CSV
import datetime

import warnings
warnings.filterwarnings("ignore")

In [42]:
#this is an input for jobs and location
occupation = str(input("Job title: "))
loc = str(input("City, State(i.e. TX): "))

### You will need to folder that the webdriver is in:

Download the webdriver here: https://www.seleniumhq.org/download/

Note the path you download the file from. Locate the file and unzip it.

In [43]:
path_to_chromedriver = "./chromedriver_win32/chromedriver.exe"

In [44]:
#setup browser window
browser = webdriver.Chrome(executable_path= path_to_chromedriver)

In [45]:
#adds the data in URL format by removing any white space and adding the data. 
def Keyword_to_url(kw):
    kw = kw.strip(" ").replace(" ","%20")
    return kw

In [46]:
# this will be used as the URL input for multiple job searches. 
radius = str("5")
page_size = "100" #how many pages would you like to get. 
source = 'IN' #stands for indeed

In [47]:
#give it the siteD
url = 'https://www.careeronestop.org/Toolkit/Jobs/find-jobs.aspx?keyword='+ \
    Keyword_to_url(occupation)+\
    '&ajax=0&location='+loc+'&radius='+radius+ \
    '&source='+ source +'&pagesize='+page_size+'&sortcolumns=accquisitiondate&sortdirections=DSC'
browser.get(url)
time.sleep(1)

In [48]:
listing_num = len(browser.find_elements_by_xpath('//*[@data-title="Job Title"]'))

In [49]:
job_links = browser.find_elements_by_xpath('//*[@data-title="Job Title"]')

In [50]:
#these catch the title of the job details. 
company = []
position = []
location = []
date_posted = []
job_url = []
pos_description = []

In [51]:
def collect_job_listings(num):
    
    """
    Works on an individual link basis.
    Creates dataframe from the link and job listed on the site. 
    the job descriptions will be added later.
    """
    
    num = int(num)
    position.append(job_links[num].text)
    company.append(browser.find_elements_by_xpath('//*[@data-title="Company"]')[num].text)
    location.append(browser.find_elements_by_xpath('//*[@data-title="Location"]')[num].text)
    date_posted.append(browser.find_elements_by_xpath('//*[@data-title="Date Posted"]')[num].text)
    job_url.append(job_links[num].find_element_by_tag_name("a").get_attribute('href'))
    
    return pd.DataFrame({'company': company , 
                     'position': position, 
                     'location':location,
                     'date_posted':date_posted,'job_url': job_url,
                     'position_description':'0'},columns=['company','location','position','date_posted','job_url','position_description'])
    

In [52]:
#loop to obtain all information for each job in the table.
for links in range(listing_num):
    collect_job_listings(links)

In [55]:
#creating a dataframe of all data collected.
job_list = pd.DataFrame({'company': company , 
                     'position': position, 
                     'location':location,
                     'date_posted':date_posted,'job_url': job_url,
                     'position_description':'0'},columns=['company','location','position','date_posted','job_url','position_description']).drop_duplicates()
#drops all duplicates and replaces them with

In [56]:
#checks for page errors and to see which ones. 
err = []

In [66]:
#Loops the urls and receives the content from the webpage.
for i in tqdm(range(job_list.shape[0])):
    time.sleep(np.random.randint(2,7))
    try:
        browser.get(job_list['job_url'][i])
        pos_des = browser.find_element_by_class_name("snip").text
        pos_des = pos_des.replace("\n",' ')
        job_list['position_description'][i] = job_list['position_description'][i].replace('0',pos_des)
    except:
        err.append(i)
        


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [08:18<00:00,  4.98s/it]


In [68]:
browser.get(job_list['job_url'][1])

In [69]:
browser.close()

In [70]:
job_list.to_csv("./data/"+str(datetime.datetime.today())[:10]+"_"+occupation+"_"+loc+" job_list.csv")